Mongolian text classification series #01

In this notebook I'm gonna try to classify cyrillic mongolian texts with word2vec initialization, facebook's pretrained word2vec weights will be used.  

Eduge dataset provided by Bolorsoft LLC

Author : Sharavsambuu Gunchinish (sharavsambuu@gmail.com)

Github: https://github.com/sharavsambuu/mongolian-text-classification 



In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-alpha0
!pip install gensim

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.0.0-alpha0


[More info about creation of eduge dataset pickles](https://github.com/sharavsambuu/mongolian-text-classification/blob/master/preprocess_dataset/preprocess_eduge.ipynb) preprocessing eats a lot of CPU cycle so it's good idea to cook it before using colab.

In [2]:
import os
from os.path import exists, join, basename, splitext
import sys

def download_from_google_drive(file_id, file_name):
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# download eduge pickles
file_path = 'eduge_pickles'
if not exists(file_path):
  download_from_google_drive('1vjJ9YgIe8o0ErhbN0lH1XqPv3KFP8acv', '%s.rar' % file_path)
  rar_file = file_path+".rar"
  !unrar x $rar_file

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   2984      0 --:--:-- --:--:-- --:--:--  2984
100  106M    0  106M    0     0   116M      0 --:--:-- --:--:-- --:--:--  116M

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from eduge_pickles.rar


Would you like to replace the existing file word_index.pickle
9178153 bytes, modified on 2019-04-13 01:44
with a new one
9178153 bytes, modified on 2019-04-13 01:44

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit q

Program aborted


In [0]:
import pickle

with open('word_index.pickle', 'rb') as handle:
  word_index = pickle.load(handle)
    
with open('reversed_word_index.pickle', 'rb') as handle:
  reversed_word_index = pickle.load(handle)
  
with open('eduge_stopwords_removed.pickle', 'rb') as handle:
  eduge_ds = pickle.load(handle)

In [4]:
# facebook trained word2vec on both commoncrawl and wikipedia. So this model should contain enough representation about our mongolian words.
mongolian_word2vec_download="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.bin.gz"
if not exists("cc.mn.300.bin.gz"):
  !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.bin.gz
  !gunzip cc.mn.300.bin.gz

--2019-04-13 07:40:10--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2937042399 (2.7G) [application/octet-stream]
Saving to: ‘cc.mn.300.bin.gz’

cc.mn.300.bin.gz    100%[===================>]   2.73G  27.9MB/s    in 1m 41s  

2019-04-13 07:41:51 (27.8 MB/s) - ‘cc.mn.300.bin.gz’ saved [2937042399/2937042399]



In [5]:
from gensim.models.wrappers import FastText

word2vec_model = FastText.load_fasttext_format('cc.mn.300.bin')

W0413 07:43:49.155202 140348932761472 ssh.py:33] paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
W0413 07:43:49.536088 140348932761472 word2vec.py:573] Slow version of gensim.models.deprecated.word2vec is being used


In [9]:
print(word2vec_model.most_similar('монгол'))
#print(word2vec_model["монгол"])

[('Монгол', 0.6342526078224182), ('монголын', 0.6047513484954834), ('хятад', 0.5558866858482361), ('Монголын', 0.5087883472442627), ('судлалаараа', 0.48851606249809265), ('манай', 0.4853793680667877), ('уйгаржин', 0.4725492596626282), ('угсаатангууд', 0.47093287110328674), ('орос', 0.46463483572006226), ('худам', 0.4609120190143585)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [12]:
# preparing embedding matrix
import numpy as np

words_not_found = []
embed_dim       = 300
embedding_matrix = np.random.uniform(-1, 1, (len(word_index), embed_dim))
for word, i in word_index.items():
  if i<4:
    continue
  try:
    embedding_vector = word2vec_model[word]
    if (embedding_vector is not None) and len(embedding_vector) > 0:
      # words not found in embedding index will be all-zeros.
      embedding_matrix[i] = embedding_vector
  except:
    words_not_found.append(word)
    pass
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

number of null word embeddings: 0


In [15]:
print(embedding_matrix.shape)
#print(embedding_matrix[5])

(370794, 300)
[-1.05637871e-02 -2.08224468e-02 -9.15830198e-04 -8.30183830e-03
  9.11077426e-04  9.29622073e-03  3.47890495e-03  7.40211690e-03
  1.29913762e-02 -2.28701718e-03  3.09971930e-03  2.61653843e-03
  9.35615599e-03 -6.32976706e-04 -1.44581655e-02  8.83297715e-03
 -6.13231072e-03  1.04775308e-02  1.19179329e-02  6.72390871e-03
  1.04715759e-02 -1.16485776e-02  8.95684352e-05 -1.05829490e-02
 -6.46159798e-03 -9.41502955e-03  9.76312812e-03 -4.39939508e-03
 -5.60690090e-03 -6.08651107e-03 -1.55586125e-02  7.06539676e-03
 -1.42321270e-03 -8.99509992e-03  1.04483729e-02  8.31610896e-03
 -4.51149972e-04 -5.54138934e-03  2.79903202e-03  5.56980027e-04
  1.65209528e-02  2.00354122e-02  7.43129430e-03 -1.37951663e-02
 -7.00565055e-04 -4.40782728e-03 -8.72603850e-04 -7.60663627e-03
  1.29072955e-02  5.86537318e-03  8.94303620e-03 -3.35041224e-03
 -5.27137145e-03 -3.46761080e-03 -9.28205252e-03  2.71191285e-03
 -1.02881091e-02  5.94624365e-03 -5.10198297e-03  1.34274065e-02
  6.2523002

In [0]:
MAX_LEN = 512

import itertools

for item in eduge_ds:
  item[0] = list(itertools.chain(*item[0]))[:MAX_LEN]

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(eduge_ds, test_size=0.1, random_state=999)

In [0]:
train_data_words  = [i[0] for i in train]
train_label_words = [i[1] for i in train]
test_data_words   = [i[0] for i in test ]
test_label_words  = [i[1] for i in test ]

In [0]:
def encode_news(text):
    return [word_index.get(i, 2) for i in text]
  
train_data = [encode_news(sent) for sent in train_data_words]
test_data  = [encode_news(sent) for sent in test_data_words ]

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAX_LEN)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=MAX_LEN)

In [21]:
labels = list(set(test_label_words))
labels

['байгал орчин',
 'улс төр',
 'хууль',
 'урлаг соёл',
 'боловсрол',
 'спорт',
 'эдийн засаг',
 'технологи',
 'эрүүл мэнд']

In [0]:
from sklearn.preprocessing import LabelBinarizer
encoder     = LabelBinarizer()
train_label = transfomed_label = encoder.fit_transform(train_label_words)
test_label  = transfomed_label = encoder.fit_transform(test_label_words )

In [24]:
vocab_size = len(word_index)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embed_dim, weights=[embedding_matrix], input_length=MAX_LEN, trainable=False))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(embed_dim, activation='relu'))
model.add(keras.layers.Dense(len(labels), activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 300)          111238200 
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               90300     
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 2709      
Total params: 111,331,209
Trainable params: 93,009
Non-trainable params: 111,238,200
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
print(len(train_data), len(train_label))
print(len(test_data ), len(test_label) )

partial_index = 3000

x_val = train_data[:partial_index]
partial_x_train = train_data[partial_index:]

y_val = train_label[:partial_index]
partial_y_train = train_label[partial_index:]

68094 68094
7567 7567


In [28]:
epochs = 150
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 65094 samples, validate on 3000 samples
Epoch 1/150
65094/65094 [==============================] - 2s 34us/sample - loss: 1.2211 - accuracy: 0.6209 - val_loss: 1.2301 - val_accuracy: 0.6160
Epoch 2/150
65094/65094 [==============================] - 2s 32us/sample - loss: 1.2091 - accuracy: 0.6277 - val_loss: 1.2222 - val_accuracy: 0.6193
Epoch 3/150
65094/65094 [==============================] - 2s 32us/sample - loss: 1.1974 - accuracy: 0.6332 - val_loss: 1.2195 - val_accuracy: 0.6423
Epoch 4/150
65094/65094 [==============================] - 2s 32us/sample - loss: 1.1884 - accuracy: 0.6338 - val_loss: 1.2081 - val_accuracy: 0.6047
Epoch 5/150
65094/65094 [==============================] - 2s 32us/sample - loss: 1.1761 - accuracy: 0.6379 - val_loss: 1.1940 - val_accuracy: 0.6207
Epoch 6/150
65094/65094 [==============================] - 2s 32us/sample - loss: 1.1666 - accuracy: 0.6414 - val_loss: 1.1870 - val_accuracy: 0.6197
Epoch 7/150
65094/65094 [==========================

In [29]:
results = model.evaluate(test_data, test_label)
print(results)

7567/7567 [==============================] - 1s 105us/sample - loss: 0.8068 - accuracy: 0.7371
[0.8067786961427623, 0.73714817]


In [52]:
data_index   = 12
data_words   = " ".join(test_data_words[data_index])
data_indexes = test_data[data_index]
print(data_words)

predicted = model.predict([[data_indexes]])
print(encoder.classes_[np.argmax(predicted)])

спортын төв ордонд өнөөдөр азийн оюутны аварга шалгаруулах эмэгтэй волейболчдын хоёр дахь удаагийн тэмцээний талаар мэдээлэл хийлээ анхны тэмцээн онд тайландын бангконг хотноо болж хоёрдугаар тэмцээнийг азийн оюутны спортын холбооноос аосх олгосон эрхийн дагуу оны дөрөвдүгээр сарын ны өдрүүдэд монгол улсын нийслэл улаанбаатар хотноо зохион байгуулах тэмцээний эрхийг монгол улс оны тавдугаар сарын хуралдсан аосхны гүйцэтгэх хорооны хурлаар хоёр оронтой өрсөлдөн авчээ уг тэмцээнийг монгол улсад авах талаар мосхолбоо оноос санаачлага гарган хөөцөлдөж эхэлсэн тэмцээний эрхийг авахад муын засгийн газрын санхүүгийн дэмжлэг мэргэжлийн холбоодын ажлын туршлага манай улсын олон улсын нэр хүнд ихээхэн тус хүргэжээ зохион байгуулах хороог с ламбаа удирдаж тэмцээний зохион байгуулах хороог збх эрүүл мэндийн сайдын оны тоот тушаалаар батлаж даргаар уихын гишүүн монголын волейболын холбооны мвх хүндэт ерөнхийлөгч сламбаа ажиллаж збхны орлогч даргаар згхагентлагбтсгын дарга чнаранбаатар збхны нарийн 